In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_tables_from_url(url):
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all table elements
        tables = soup.find_all('table')
        
        # Initialize a counter for naming CSV files
        table_count = 0
        
        # Loop through each table and save it as a CSV file
        for table in tables:
            # Convert the table to a DataFrame
            df = pd.read_html(str(table))[0]
            
            # Generate a file name
            file_name = f"table_{table_count}.csv"
            
            # Save the DataFrame to a CSV file
            df.to_csv(file_name, index=False)
            print(f"Saved {file_name}")
            
            # Increment the counter
            table_count += 1
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

# URL of the page to scrape
url = 'https://fbref.com/en/players/7aa8adfe/scout/365_m1/Alejandro-Garnacho-Scouting-Report'

# Call the function with the specified URL
scrape_tables_from_url(url)
